In [1]:
import os
import glob
import numpy as np
from obspy import Stream, Trace, UTCDateTime
import seisbench.models as sbm
import csv
from model import UNet2D
from scipy.signal import ShortTimeFFT
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:
# === パラメータ ===
SAMPLE_RATE = 100
WIN_LENGTH = 30
HOP_LENGTH = 15
N_FFT = 60
EPS = 1e-8

SFT = ShortTimeFFT(
    win=np.hanning(WIN_LENGTH),
    hop=HOP_LENGTH,
    fs=SAMPLE_RATE,
    fft_mode="onesided2X",
    mfft=N_FFT,
    scale_to="magnitude",
)

# === モデル読み込み ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet2D()
model.load_state_dict(torch.load("model/best_model.pt", map_location=device))
model = model.to(device)
model.eval()


def denoise(model, deepdenoiser, original_stream):
    deepdenoised_stream = deepdenoiser.annotate(original_stream)
    deepdenoised = convert_stream_to_ndarray(deepdenoised_stream, channel_order=["DeepDenoiser_UD", "DeepDenoiser_NS", "DeepDenoiser_EW"])
    deepdenoised_abs = np.abs(SFT.stft(deepdenoised))

    original = convert_stream_to_ndarray(original_stream)  # shape: [3, N]
    spec_list = []
    orig_mag_list = []
    denoised_mag_list = []

    for i in range(3):
        Zxx = SFT.stft(original[i])
        real = np.real(Zxx)
        imag = np.imag(Zxx)
        spec_list.append(real)
        spec_list.append(imag)
        orig_mag_list.append(np.abs(Zxx))

    spec_array = np.stack(spec_list, axis=0).astype(np.float32)  # shape: [6, F, T]

    for i in range(3):
        real = spec_array[2*i]
        imag = spec_array[2*i+1]
        
        mean_real = np.mean(real)
        std_real = np.std(real) + 1e-6
        spec_array[2*i] = (real - mean_real) / std_real

        mean_imag = np.mean(imag)
        std_imag = np.std(imag) + 1e-6
        spec_array[2*i+1] = (imag - mean_imag) / std_imag

    # 推論
    spec_tensor = torch.from_numpy(spec_array).float().unsqueeze(0).to(device)  # [1, 6, F, T]
    with torch.no_grad():
        pred_mask = model(spec_tensor).squeeze(0).cpu().numpy()  # [3, F, T]

    # real・imag にマスクをかけて再構成
    denoised = []
    for i in range(3):
        Zxx = SFT.stft(original[i])
        denoised_Zxx = Zxx * pred_mask[i]
        wav = SFT.istft(denoised_Zxx, k1=3000)
        denoised_mag_list.append(np.abs(denoised_Zxx))

        denoised.append(wav.astype(np.float32))

    return np.stack(denoised), np.array(orig_mag_list), np.array(denoised_mag_list), deepdenoised, deepdenoised_abs  # shape: [3, N]


# Utility functions
def convert_ndarry_stream(data, time_str, station_name, sampling_rate=100):
    """
    Convert a 2D NumPy array of waveform data into an ObsPy Stream object.

    Parameters
    ----------
    data : np.ndarray
        2D array of shape (3, N), where N is the number of samples for each component (UD, NS, EW).
    time_str : str
        Start time string in the format 'yymmdd_HHMMSS' (e.g., '150323_141948').
    station_name : str
        Station name to be assigned to each trace.
    sampling_rate : float, optional
        Sampling rate in Hz. Default is 100 Hz.

    Returns
    -------
    stream : obspy.Stream
        ObsPy Stream object containing three Traces with appropriate metadata.
    """
    year = 2000 + int(time_str[:2])
    month, day = int(time_str[2:4]), int(time_str[4:6])
    hour, minute, second = int(time_str[7:9]), int(time_str[9:11]), int(time_str[11:13])
    utc_time = UTCDateTime(year, month, day, hour, minute, second)

    channels = ["UD", "NS", "EW"]
    stream = Stream()
    for i, ch in enumerate(channels):
        trace = Trace(data=data[i, :])
        trace.stats.update({
            "sampling_rate": sampling_rate,
            "starttime": utc_time,
            "network": "MeSO-net",
            "station": station_name,
            "location": "",
            "channel": ch,
        })
        stream.append(trace)
    return stream

def convert_stream_to_ndarray(stream, channel_order=["UD", "NS", "EW"]):
    """
    Convert ObsPy Stream to ndarray of shape (samples, channels).

    Parameters:
        stream (obspy.Stream): Stream object containing 3 components.
        channel_order (list): Order of channels to extract, default is ["UD", "NS", "EW"].

    Returns:
        np.ndarray: Array of shape (3, n_samples)
    """
    traces = []
    for ch in channel_order:
        tr = stream.select(channel=ch)
        if len(tr) == 0:
            raise ValueError(f"Channel {ch} not found in the stream.")
        traces.append(tr[0].data)

    # Stack and transpose to shape (samples, channels)
    data = np.stack(traces)
    return data

def calc_snr(signal, noise):
    """
    Calculate signal-to-noise ratio (SNR) in decibels (dB).

    Parameters:
    ----------
    signal : np.ndarray
        Array containing the signal portion.
    noise : np.ndarray
        Array containing the noise portion.

    Returns:
    -------
    float
        SNR value in dB.
    """
    return 10 * np.log10(np.std(signal) / np.std(noise))

def calc_cc(a, b):
    """
    Calculate the Pearson correlation coefficient between two signals.

    Parameters:
    ----------
    a : np.ndarray
        First signal.
    b : np.ndarray
        Second signal.

    Returns:
    -------
    float
        Correlation coefficient between a and b (range: -1 to 1).
    """
    return np.corrcoef(a, b)[0, 1]

def zscore(data):
    """
    Normalize each channel using z-score normalization (zero mean, unit variance).

    Parameters:
    ----------
    data : np.ndarray
        2D array of shape (channels, time), e.g., (3, N).

    Returns:
    -------
    np.ndarray
        Z-score normalized data with the same shape.
    """
    mean = np.mean(data, axis=1, keepdims=True)
    std = np.std(data, axis=1, keepdims=True)
    normalized_data = (data - mean) / std
    return normalized_data

def calc_loss(data1, data2, p_onset, s_onset, sf=100):

    '''
    Calculate a loss value based on signal-to-noise ratio (SNR) and correlation coefficients (CC)
    between original and denoised seismic waveform data for P-wave, S-wave, and noise segments.

    Parameters:
    ----------
    data1 (Original wave) : np.ndarray
        Original waveform data of shape (3, N), where N is the number of time steps.
    data2 (Denoised wave) : np.ndarray
        Denoised waveform data of shape (3, N), corresponding to data1.

    Returns:
    -------
    loss : float
        Averaged loss across 3 channels, combining SNR and CC values.
    P_SNR : float
        Averaged P-wave SNR after denoising.
    S_SNR : float
        Averaged S-wave SNR after denoising.
    P_CC : float
        Averaged correlation coefficient between original and denoised P-wave signals.
    s_cc : float
        Averaged correlation coefficient between original and denoised S-wave signals.
    n_cc : float
        Averaged correlation coefficient between original and denoised noise segments.
    '''

    p_snrs = []
    s_snrs = []
    p_ccs = []
    s_ccs = []
    n_ccs = []

    loss = []

    for ch in [0,1,2]:
        orig_data = data1[ch,:]
        den_data = data2[ch,:]

        signal_p = orig_data[p_onset : p_onset+sf*5]
        noise_p = orig_data[p_onset-sf*5 : p_onset]

        signal_p_deno = den_data[p_onset : p_onset+sf*5]
        noise_p_deno = den_data[p_onset-sf*5 : p_onset]

        signal_s = orig_data[s_onset : s_onset+sf*5]
        signal_s_deno = den_data[s_onset : s_onset+sf*5]

        snr_p_orig = calc_snr(signal_p, noise_p)
        snr_s_orig = calc_snr(signal_s, noise_p)

        snr_p_deno = calc_snr(signal_p_deno, noise_p_deno)
        snr_s_deno = calc_snr(signal_s_deno, noise_p_deno)

        cc_n = calc_cc(noise_p, noise_p_deno)
        cc_p = calc_cc(signal_p, signal_p_deno)
        cc_s = calc_cc(signal_s, signal_s_deno)

        loss_ch = (snr_p_deno + snr_s_deno) * cc_p * cc_s * cc_n

        p_snrs.append(snr_p_deno)
        s_snrs.append(snr_s_deno)

        p_ccs.append(cc_p)
        s_ccs.append(cc_s)
        n_ccs.append(cc_n)

        loss.append(loss_ch)

    return np.mean(loss), p_snrs, s_snrs, p_ccs, s_ccs, n_ccs

In [ ]:
# Load pretrained denoising model
deepdenoiser = sbm.DeepDenoiser.from_pretrained("original")



model_type = "CBAM"
total_loss = 0

# Prepare data
files = sorted(glob.glob('data/Test/*'))     # 適切なパスを設定

print(len(files))

with open(f'reports/{model_type}_loss_results.csv', 'w', newline='') as csvfile:   # 適切なパスを設定
    writer = csv.writer(csvfile)

    # headder
    writer.writerow(['FileName', 'LOSS', 'UD_P_SNR', 'UD_S_SNR', 'UD_P_CC', 'UD_S_CC', 'UD_N_CC', 'NS_P_SNR', 'NS_S_SNR', 'NS_P_CC', 'NS_S_CC', 'NS_N_CC', 'EW_P_SNR', 'EW_S_SNR', 'EW_P_CC', 'EW_S_CC', 'EW_N_CC'])

    for fn in tqdm(files, total=len(files)):
      data = np.load(fn)
      wave, p_onset, s_onset = data['wave'], data['pidx'], data['sidx']
      time_str, station_name = os.path.basename(fn).replace('.npz', '').split('_')

      # Create ObsPy Stream
      original_stream = convert_ndarry_stream(wave-np.mean(wave, axis=1, keepdims=True), time_str, station_name)

      denoised, orig_abs, denoised_abs, deepdenoised, deepdenoised_abs = denoise(model, deepdenoiser, original_stream)
      original = convert_stream_to_ndarray(original_stream, channel_order=["UD", "NS", "EW"])

      if model_type == "deepdenoiser":
          denoised = deepdenoised
    #   elif model_type == "CBAM":
    #       titles = ["UD", "NS", "EW"]
    #       fig, axes = plt.subplots(6, 3, figsize=(16, 20))

    #       for i in range(3):
    #           axes[0, i].set_title(f"Original {titles[i]}")
    #           axes[0, i].plot(np.linspace(0, 30, 3000), original[i])
    #           axes[0, i].set_xlim(0, 30)
    #           axes[1, i].imshow(np.log10(orig_abs[i] + 1e-8), extent=[0, 30, 0, 50], aspect="auto", origin="lower")

    #           axes[2, i].set_title(f"Denoised {titles[i]}")
    #           axes[2, i].plot(np.linspace(0, 30, 3000), denoised[i])
    #           axes[2, i].set_xlim(0, 30)
    #           axes[3, i].imshow(np.log10(denoised_abs[i] + 1e-8), extent=[0, 30, 0, 50], aspect="auto", origin="lower", vmin=0, vmax=1)

    #           axes[4, i].set_title(f"DeepDenoiser {titles[i]}")
    #           axes[4, i].plot(np.linspace(0, 30, 3000),deepdenoised[i])
    #           axes[4, i].set_xlim(0, 30)
    #           axes[5, i].imshow(np.log10(deepdenoised_abs[i] + 1e-8), extent=[0, 30, 0, 50], aspect="auto", origin="lower", vmin=0, vmax=1)


    #       # 縦の間隔を広げる
    #       plt.subplots_adjust(hspace=0.4)

    #       # ファイル名を動的に生成して保存
    #       fig_name = f"figures/{os.path.basename(fn).replace('.npz', '')}_comparison.png"
    #       os.makedirs("figures", exist_ok=True)
    #       plt.savefig(fig_name, dpi=300, bbox_inches='tight')
    #       plt.close()
          
      
  

      loss, p_snrs, s_snrs, p_ccs, s_ccs, n_ccs = calc_loss(original, denoised, p_onset, s_onset)

      writer.writerow([os.path.basename(fn),
                       loss,
                       p_snrs[0], s_snrs[0], p_ccs[0], s_ccs[0], n_ccs[0],
                       p_snrs[1], s_snrs[1], p_ccs[1], s_ccs[1], n_ccs[1],
                       p_snrs[2], s_snrs[2], p_ccs[2], s_ccs[2], n_ccs[2]])

      print(os.path.basename(fn), loss, p_snr, s_snr, P_CC, s_cc, n_cc)

      total_loss += loss

print(f'Total Loss = {total_loss/(len(files))}')


500


100%|██████████| 500/500 [12:07<00:00,  1.45s/it]

Total Loss = 18.343057021001176


In [ ]:
import pandas as pd

df = pd.read_csv("mine_loss_results.csv")
df.describe()

,LOSS,UD_P_SNR,UD_S_SNR,UD_P_CC,UD_S_CC,UD_N_CC,NS_P_SNR,NS_S_SNR,NS_P_CC,NS_S_CC,NS_N_CC,EW_P_SNR,EW_S_SNR,EW_P_CC,EW_S_CC,EW_N_CC
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,12.580729,15.804465,16.050645,0.959767,0.977638,0.354823,19.502171,24.291604,0.947203,0.984400,0.375697,19.163827,23.676196,0.945870,0.983163,0.376412
std,6.542047,6.295686,6.991246,0.068830,0.042132,0.180425,6.867532,7.268912,0.077355,0.034436,0.217081,6.573299,7.045687,0.073924,0.036073,0.217266
min,0.372592,-1.348372,-3.355258,0.424526,0.570517,0.091079,-1.535250,0.789417,0.487167,0.522748,0.079339,-0.575507,1.164487,0.585403,0.628441,0.102688
25%,8.182358,11.819522,11.025746,0.952316,0.976396,0.234248,15.217295,19.705909,0.932430,0.984817,0.217800,14.901349,19.004151,0.928401,0.983647,0.215934
50%,10.775206,15.299281,15.784638,0.985888,0.992890,0.302857,19.781446,24.432595,0.976593,0.996120,0.293704,19.163641,23.669145,0.978109,0.995575,0.295649
75%,15.222834,19.306071,20.381430,0.997365,0.998668,0.413846,23.428479,29.171625,0.995210,0.998819,0.479068,22.895944,28.135418,0.995423,0.998634,0.486489
max,44.169810,38.856273,40.124755,0.999896,0.999901,0.977706,45.011053,48.377485,0.999876,0.999875,0.971214,42.478752,44.091229,0.999880,0.999916,0.969519


In [ ]:
[]